# 1. SVM (6k)

## 1.1. Revisión de la base de datos

Con la BBDD que me compartió Carlos, voy a crear un modelo que prediga la calidad de inmuebles a partir de la descripción de venta. Ojo: no uso los datos de Paúl porque no tienen el resultado factorial de Carlos: *Estado_contemporaneidad_calidad* (FAC1_1) y *Ausencia_singulares_presencia_arm_cocina* (FAC2_1).

In [1]:
import pandas as pd
import numpy as np
import pyreadstat

Importamos el archivo, revisamos las columnas, nos quedamos sólo con las que nos interesan y vemos cuántas filas tenemos.

In [2]:
h=pd.read_spss('rawdata/BDDHabitaclia_4043_join.sav')

In [3]:
print(h.columns.tolist())

['OBJECTID_1', 'codigo_inmueble1', 'Title', 'Type_build', 'Type_opera', 'Link', 'Location', 'Lat_X', 'Lon_Y', 'Climatic_Z', 'Nom_Mun', 'precio_eur', 'superficie', 'superficie_2', 'Unit_price', 'Ln_total_pr', 'Ln_unit_pr', 'numero_habitaciones', 'numero_bano', 'ratio_bano_hab', 'numero_aseo', 'ascensor', 'interac_planta', 'numero_de_piso', 'anyo_constr', 'anyo_constr_ponderad', 'antig_ponderad', 'Inverse_Age', 'Year_Before_1981', 'Year_1982_2006', 'Year_After_2007', 'superficie_terraza_m2', 'grand_terr_20m2', 'superficie_jardin_m2', 'superficie_salon', 'bool_despacho', 'bool_buhardilla', 'bool_trastero', 'bool_lavadero', 'bool_piscina_comunitaria', 'bool_jardin_comunitario', 'bool_amueblado', 'bool_ascensor', 'descripcion', 'bool_aire_acondicionado', 'bool_calefaccion', 'bool_chimenea', 'texto_destacado', 'Description', 'calificacion_consumo_letra', 'calificacion_consumo_valor', 'calificacion_emision_letra', 'calificacion_emision_valor', 'Dum_EPC', 'EPC_A_emision', 'EPC_B_emision', 'EPC

Si queremos saber si hay alguna celda vacía en nuestro dataframe, ejecutamos el siguiente código.

In [64]:
h.isnull().values.any()

True

Como ya sabemos que sí, las eliminamos.

In [67]:
h = h.dropna()

## 1.2. Definimos la(s) variable(s) a explicar

Voy a buscar qué columnas son dummies.

In [68]:
unique_counts = h.nunique()
categoricas = unique_counts[unique_counts < 5].index
categoricas

Index(['Type_build', 'Type_opera', 'Climatic_Z', 'numero_aseo', 'ascensor',
       'Year_Before_1981', 'Year_1982_2006', 'Year_After_2007',
       'grand_terr_20m2', 'superficie_jardin_m2', 'bool_despacho',
       'bool_buhardilla', 'bool_trastero', 'bool_lavadero',
       'bool_piscina_comunitaria', 'bool_jardin_comunitario', 'bool_amueblado',
       'bool_ascensor', 'bool_aire_acondicionado', 'bool_calefaccion',
       'bool_chimenea', 'Dum_EPC', 'EPC_A_emision', 'EPC_B_emision',
       'EPC_C_emision', 'EPC_D_emision', 'EPC_E_emision', 'EPC_F_emision',
       'EPC_G_emision', 'dum_acces_viappal', 'calidad_cocina', 'diseny_cocina',
       'alta_calidad', 'reform_inmob', 'dum_mar_200m', 'dum_ttpp_riel_urb',
       'C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 'B_estado',
       'B_lavamano', 'R_contempo', 'R_estado', 'R_carpinte', 'R_singular',
       'R_ventana', 'Dum_precio', 'scrap_year', 'filter_$', 'Muestra_2023',
       'EPC_A_emision_2023', 'EPC_B_emision_2023', 'EPC_C_

Noto que desde `C_contempo` hasta `R_ventana` tenemos datos de calidad arquitectónica, por lo que el primer paso será hacer un Análisis de Componentes Principales para disminuir en la medida de la posible estas variables.

In [69]:
from factor_analyzer import FactorAnalyzer #librería especializada para hacer ACP

#Le indicamos en qué columnas se va a centrar
df = h[['C_contempo','C_estado','C_armarios','B_contempo','B_estado', 'B_lavamano',
           'R_contempo','R_estado','R_singular','R_ventana']]

#Le pedimos un ACP con rotación varimax y dos factores (esto ya viene de un análisis de Carlos
#El nª de factores se debe analizar previamente.
fa = FactorAnalyzer(rotation='varimax', n_factors=2, method='principal')
fa.fit(df)

rotated_loadings = fa.loadings_

# Transponer la matriz de cargas rotadas para que las columnas coincidan con las variables
rotated_loadings_transposed = rotated_loadings.T

rotated_df = pd.DataFrame(rotated_loadings_transposed, columns=df.columns)

rotated_df

,C_contempo,C_estado,C_armarios,B_contempo,B_estado,B_lavamano,R_contempo,R_estado,R_singular,R_ventana
0,0.771175,0.829585,0.216031,0.812078,0.827633,0.524474,0.782858,0.800836,0.350748,0.330651
1,-0.087311,0.175826,0.744095,-0.111461,0.138680,0.140660,-0.103330,0.144193,-0.551318,0.285422


Revisando los datos, me doy cuenta que `0` se me acerca a medir el *estado de contemporaneidad y calidad*, mientras que `1`, a medir la *ausencia de elementos singulares*. Los añadimos al dataframe original.

In [73]:
# Obtener los factores extraídos
factors = fa.transform(df)

# Agregar los factores al DataFrame original
h['Estado_contemporaneidad_calidad'] = factors[:, 0]
h['Ausencia_singulares_presencia_arm_cocina'] = factors[:, 1]

## 1.3. Definimos la variable explicativa

Ahora voy a buscar qué columnas tienen valores *string*, es decir, letras.

In [76]:
colstring = h.select_dtypes(include=['object'])
print(colstring.columns)

Index(['Title', 'Type_build', 'Type_opera', 'Link', 'Location', 'Climatic_Z',
       'Nom_Mun', 'descripcion', 'texto_destacado', 'Description',
       'calificacion_consumo_letra', 'calificacion_emision_letra', 'persona'],
      dtype='object')


Reviso a mayor profundidad qué información tienen aquellas que, creo, pueden acercarme a saber el anuncio de venta.

Ahora que ya sé qué columnas me interesan, las selecciono y empiezo a trabajar.

In [6]:
colstring[['texto_destacado','Description']]

,texto_destacado,Description
0,4 habitaciones en 11 de setembre,"Piso reformado de 4 habitaciones, salón comedo..."
1,PIS BENET MATEU/ MANUEL DE FALLA,"BENET MATEU, PIS D´ORIGEN AMB MOLT BONA DISTRI..."
2,Apartamento tipo casa Canyelles,Apartamento pero con acceso independiente desd...
3,TODO EXTERIOR Y REFORMADO,"[A2977]PISAZO, EL MEJOR DE LA ZONA.FENOMENAL P..."
4,102 M2 EXTERIORES CON ASCENSOR,[A3001]VIVIENDA EN LA CALLE GARROFER DE SANT I...
...,...,...
5462,Piso en venta en Can Pantiquet-Riera Seca,Piso En Mollet Del Vallès!Ubicado a 600m de la...
5463,BUENA UBICACIÓN!,"La Casa Agency presenta en Exclusividad, esta ..."
5464,PISO EN PLANTA BAJA CON PATIO DE 60m²,Mis Finques promociona esta planta baja con pa...
5465,PISO CON TERRAZA,PISO CON TERRAZAPiso con TERRAZA DE 40M2. La v...


Ahora ya sé que me interesan `Description`, `Estado_contemporaneidad_calidad` y `Ausencia_singulares_presencia_arm_cocina`.

## 1.4. Modelo `Estado_contemporaneidad_calidad`

Elaboro el modelo.

In [87]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
import nltk
from nltk.stem import WordNetLemmatizer
import unicodedata

# Descargar recursos necesarios para NLTK
nltk.download('punkt')
nltk.download('wordnet')

# Definir función para lematización
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

# Definir función para eliminar tildes y convertir a minúsculas
def preprocess_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = ''.join(char for char in unicodedata.normalize('NFD', text) if unicodedata.category(char) != 'Mn')  # Eliminar tildes
    return text

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(h['Description'], h['Estado_contemporaneidad_calidad'], 
                                                    test_size=0.2, 
                                                    random_state=42) #Ojo: elijo 80-20.

# Preprocesamiento y representación de texto
custom_stopwords = ['de', 'la', 'el', 'los', 'las', 'en', 'para', 
                    'con', 'y', 'o', 'un', 'una', 'que', 'se', 'su', 'sus']  # Agrega más palabras si es necesario
tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stopwords)

# Lematizar y preprocesar el texto
X_train_lemmatized = X_train.apply(lemmatize_text)
X_test_preprocessed = X_test.apply(preprocess_text)
X_test_lemmatized = X_test_preprocessed.apply(lemmatize_text)

# Entrenar el vectorizador TF-IDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_lemmatized)
X_test_tfidf = tfidf_vectorizer.transform(X_test_lemmatized)

# Obtener nombres de características y mapearlos a sus índices
feature_names = tfidf_vectorizer.get_feature_names_out()
feature_index_map = {word: idx for idx, word in enumerate(feature_names)}

# Definir diccionario de palabras y factores de multiplicación de peso (eliminando tildes)
word_weight_dict = {'noble': 2, 'reformado': 2, 'rehabilitado': 2, 'amplio': 2, 
                    'equipado': 2, 'restaurada': 2, 'isla': 2, 'isleta': 2, 'estrenar': 2,
                    'conservado': 2, 'moderna': 2, 'acondicion': 2, 'lujo': 2, 'muy': 2,
                    'diseno': 2, 'estado': 2, 'grand': 2, 'excelente': 2, 'vitroceramica': 2,
                    'renovado': 2, 'impecable': 2, 'nuevo': 2, 'perfecto': 2, 'totalmente': 2,
                    'total': 2, 'full': 2, 'espectacular': 2, 'maravillosa': 2, 'estupendo': 2,
                    'optimo': 2, 'magnifica': 2, 'ideal': 2, 'fantastica': 2, 'vanguardia': 2,
                    'office': 2, 'americana': 2, 'abierto': 2, 'vistas': 2, 'luminoso': 2,
                    'iluminada': 2, 'bien': 2, 'entrar': 2, 'actualizado': 2, 'conservacion': 2,
                    'buen': 2, 'condicion': 2, 'mantenida': 2, 'cuidado': 2, 'precioso': 2,
                    'bonita': 2, 'encanto': 2, 'magnifica/co': 2, 'senoral': 2, 'vista': 2,
                    'panoramico': 2, 'exterior': 2, 'modernista': 2, 'acogedora': 2, 'regia': 2,
                    'gusto': 2, 'noucentista': 2, 'exclusivo': 2, 'neo-clasica': 2,
                    'neoclasica': 2, 'inmejorable': 2, 'fabuloso': 2, 'majestuosa': 2,
                    'alta calidad': 2, 'alto standing': 2, 'super': 2, 'impresionante': 2,
                    'elegante': 2, 'esplendido': 2, 'calida': 2, 'especial': 2}


# Modificar pesos según el diccionario
for word, weight_factor in word_weight_dict.items():
    word_no_accents = preprocess_text(word)
    if word_no_accents in feature_index_map:
        word_index = feature_index_map[word_no_accents]
        X_train_tfidf[:, word_index] *= weight_factor
        X_test_tfidf[:, word_index] *= weight_factor

# Entrenar un modelo de regresión (por ejemplo, SVR)
svm_regressor = SVR(kernel='linear')
svm_regressor.fit(X_train_tfidf, y_train)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuari\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuari\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


SVR(kernel='linear')

In [88]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Predecir en el conjunto de prueba
y_predSVM = svm_regressor.predict(X_test_tfidf)

# Coeficiente de Determinación (R^2)
r2SVM = r2_score(y_test, y_predSVM)
# Error Absoluto Medio (MAE)
maeSVM = mean_absolute_error(y_test, y_predSVM)
# Error Absoluto Relativo (RAE)
mean_y_test = y_test.mean()
raeSVM = maeSVM / mean_y_test
# Raíz del Error Cuadrático Medio (RMSE)
rmseSVM = np.sqrt(mean_squared_error(y_test, y_predSVM))
# Raíz del Error Cuadrático Medio Relativo (RRSE)
std_y_test = y_test.std()
rrseSVM = rmseSVM / std_y_test


print("Coefficient of Determination (R^2):", r2SVM)
print("Mean Absolute Error (MAE):", maeSVM)
print("Relative Absolute Error (RAE):", raeSVM)
print("Root Mean Squared Error (RMSE):", rmseSVM)
print("Root Relative Squared Error (RRSE):", rrseSVM)

Coefficient of Determination (R^2): 0.29755162026480875
Mean Absolute Error (MAE): 0.7034394507861268
Relative Absolute Error (RAE): 111.62444628676472
Root Mean Squared Error (RMSE): 0.8540028444553915
Root Relative Squared Error (RRSE): 0.8375775242400891


## 1.5. Modelo `Ausencia_singulares_presencia_arm_cocina`

In [83]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
import nltk
from nltk.stem import WordNetLemmatizer
import unicodedata

# Descargar recursos necesarios para NLTK
nltk.download('punkt')
nltk.download('wordnet')

# Definir función para lematización
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

# Definir función para eliminar tildes y convertir a minúsculas
def preprocess_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = ''.join(char for char in unicodedata.normalize('NFD', text) if unicodedata.category(char) != 'Mn')  # Eliminar tildes
    return text

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train2, y_test2 = train_test_split(h['Description'], h['Ausencia_singulares_presencia_arm_cocina'], 
                                                    test_size=0.2, 
                                                    random_state=42) #Ojo: elijo 80-20.

# Preprocesamiento y representación de texto
custom_stopwords = ['de', 'la', 'el', 'los', 'las', 'en', 'para', 
                    'con', 'y', 'o', 'un', 'una', 'que', 'se', 'su', 'sus']  # Agrega más palabras si es necesario
tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stopwords)

# Lematizar y preprocesar el texto
X_train_lemmatized = X_train.apply(lemmatize_text)
X_test_preprocessed = X_test.apply(preprocess_text)
X_test_lemmatized = X_test_preprocessed.apply(lemmatize_text)

# Entrenar el vectorizador TF-IDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_lemmatized)
X_test_tfidf = tfidf_vectorizer.transform(X_test_lemmatized)

# Obtener nombres de características y mapearlos a sus índices
feature_names = tfidf_vectorizer.get_feature_names_out()
feature_index_map = {word: idx for idx, word in enumerate(feature_names)}

# Definir diccionario de palabras y factores de multiplicación de peso (eliminando tildes)
word_weight_dict = {'noble': 2, 'reformado': 2, 'rehabilitado': 2, 'amplio': 2, 
                    'equipado': 2, 'restaurada': 2, 'isla': 2, 'isleta': 2, 'estrenar': 2,
                    'conservado': 2, 'moderna': 2, 'acondicion': 2, 'lujo': 2, 'muy': 2,
                    'diseno': 2, 'estado': 2, 'grand': 2, 'excelente': 2, 'vitroceramica': 2,
                    'renovado': 2, 'impecable': 2, 'nuevo': 2, 'perfecto': 2, 'totalmente': 2,
                    'total': 2, 'full': 2, 'espectacular': 2, 'maravillosa': 2, 'estupendo': 2,
                    'optimo': 2, 'magnifica': 2, 'ideal': 2, 'fantastica': 2, 'vanguardia': 2,
                    'office': 2, 'americana': 2, 'abierto': 2, 'vistas': 2, 'luminoso': 2,
                    'iluminada': 2, 'bien': 2, 'entrar': 2, 'actualizado': 2, 'conservacion': 2,
                    'buen': 2, 'condicion': 2, 'mantenida': 2, 'cuidado': 2, 'precioso': 2,
                    'bonita': 2, 'encanto': 2, 'magnifica/co': 2, 'senoral': 2, 'vista': 2,
                    'panoramico': 2, 'exterior': 2, 'modernista': 2, 'acogedora': 2, 'regia': 2,
                    'gusto': 2, 'noucentista': 2, 'exclusivo': 2, 'neo-clasica': 2,
                    'neoclasica': 2, 'inmejorable': 2, 'fabuloso': 2, 'majestuosa': 2,
                    'alta calidad': 2, 'alto standing': 2, 'super': 2, 'impresionante': 2,
                    'elegante': 2, 'esplendido': 2, 'calida': 2, 'especial': 2}


# Modificar pesos según el diccionario
for word, weight_factor in word_weight_dict.items():
    word_no_accents = preprocess_text(word)
    if word_no_accents in feature_index_map:
        word_index = feature_index_map[word_no_accents]
        X_train_tfidf[:, word_index] *= weight_factor
        X_test_tfidf[:, word_index] *= weight_factor

# Entrenar un modelo de regresión (por ejemplo, SVR)
svm_regressor2 = SVR(kernel='linear')
svm_regressor2.fit(X_train_tfidf, y_train2)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuari\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuari\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


SVR(kernel='linear')

In [86]:
# Predecir en el conjunto de prueba
y_predSVM2 = svm_regressor2.predict(X_test_tfidf)

# Coeficiente de Determinación (R^2)
r2SVM2 = r2_score(y_test2, y_predSVM)
# Error Absoluto Medio (MAE)
maeSVM2 = mean_absolute_error(y_test2, y_predSVM2)
# Error Absoluto Relativo (RAE)
mean_y_test2 = y_test2.mean()
raeSVM2 = maeSVM2 / mean_y_test2
# Raíz del Error Cuadrático Medio (RMSE)
rmseSVM2 = np.sqrt(mean_squared_error(y_test2, y_predSVM2))
# Raíz del Error Cuadrático Medio Relativo (RRSE)
std_y_test2 = y_test2.std()
rrseSVM2 = rmseSVM2 / std_y_test2


print("Coefficient of Determination (R^2):", r2SVM2)
print("Mean Absolute Error (MAE):", maeSVM2)
print("Relative Absolute Error (RAE):", raeSVM2)
print("Root Mean Squared Error (RMSE):", rmseSVM2)
print("Root Relative Squared Error (RRSE):", rrseSVM2)

Coefficient of Determination (R^2): -0.02058770542835453
Mean Absolute Error (MAE): 0.7353649888431517
Relative Absolute Error (RAE): -74.10091606555228
Root Mean Squared Error (RMSE): 1.0292859489534427
Root Relative Squared Error (RRSE): 1.009585195439135


Algunos apuntes: 

- Precisión (Precision): La precisión se refiere a la proporción de las instancias clasificadas como positivas que son realmente positivas.Se calcula como el número de verdaderos positivos dividido por la suma de verdaderos positivos y falsos positivos. **Es útil cuando el costo de los falsos positivos es alto y deseas minimizarlos**.

- Exactitud (Accuracy): La exactitud es la proporción de todas las predicciones que son correctas. Se calcula como la suma de verdaderos positivos y verdaderos negativos dividido por el total de instancias. Es una medida global del rendimiento del modelo y **es útil cuando todas las clases tienen una importancia similar**.

Guardamos.

# 2. Redes neuronales

## 2.1. Modelo `Estado_contemporaneidad_calidad`

Esta vez usaremos una red neuronal con tres capas densas, cada una seguida de una capa de abandono (dropout) para evitar el sobreajuste. Utilizamos la función de activación 'relu' en las capas ocultas. El optimizador Adam se utiliza para minimizar la pérdida de entropía cruzada binaria, y la exactitud (*accuracy*) se utiliza como métrica de evaluación.

In [104]:
#from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.preprocessing import StandardScaler
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Dropout
#from tensorflow.keras.optimizers import Adam

##### Esto ya lo tengo del modelo anterior, por eso no lo ejecuto
# Dividir los datos en características (X) y etiquetas (y)
#X = habit['texto_destacado']
#y = habit['alta_calidad']

# Dividir los datos en conjuntos de entrenamiento y prueba
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Representación TF-IDF de las características de texto
#tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stopwords)
#X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
#X_test_tfidf = tfidf_vectorizer.transform(X_test)

##### A partir de aquí ya son cosas nuevas para este modelo de redes neuronales
# Escalar las características numéricas
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train_tfidf.toarray())
#X_test_scaled = scaler.transform(X_test_tfidf.toarray())

# Crear el modelo de redes neuronales
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_tfidf.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1) #Sigmoid es más adecuado para variables binarias
])

# Compilar el modelo
model.compile(optimizer=Adam(lr=0.005), loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
historyANN = model.fit(X_train_scaled, y_train, #Son las mismas del modelo anterior
                    epochs=500, batch_size=32, validation_data=(X_test_scaled, y_test),
                    verbose=False)

Encuentro los parámetros para comparar con modelo SVM.

In [105]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Hacer predicciones con el modelo entrenado
y_predANN = model.predict(X_train_tfidf)

# Calcular Coefficient of Determination (R2)
r2ANN = r2_score(y_test, y_predANN)

# Calcular Mean Absolute Error (MAE)
maeANN = mean_absolute_error(y_test, y_predANN)

# Imprimir los resultados
print("Coefficient of Determination (R2):", r2ANN)
print("Mean Absolute Error (MAE):", maeANN)

97/97 [==============================] - 0s 2ms/step


ValueError: Found input variables with inconsistent numbers of samples: [770, 3076]

Es tan malo que el R2 sale ¡negativo!

In [100]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

##### Esto ya lo tengo del modelo anterior, por eso no lo ejecuto
# Dividir los datos en características (X) y etiquetas (y)
#X = habit['texto_destacado']
#y = habit['alta_calidad']

# Dividir los datos en conjuntos de entrenamiento y prueba
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Representación TF-IDF de las características de texto
#tfidf_vectorizer = TfidfVectorizer(stop_words=custom_stopwords)
#X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
#X_test_tfidf = tfidf_vectorizer.transform(X_test)

##### A partir de aquí ya son cosas nuevas para este modelo de redes neuronales
# Escalar las características numéricas
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train_tfidf.toarray())
#X_test_scaled = scaler.transform(X_test_tfidf.toarray())

# Crear el modelo de redes neuronales
model2 = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1) #Sigmoid es más adecuado para variables binarias
])

# Compilar el modelo
model2.compile(optimizer=Adam(lr=0.005), loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
historyANN2 = model.fit(X_train_scaled, y_train2, #Son las mismas del modelo anterior
                    epochs=500, batch_size=32, validation_data=(X_test_scaled, y_test),
                    verbose=False)

# Hacer predicciones con el modelo entrenado
y_predANN2 = model2.predict(X_test_scaled)

# Calcular Coefficient of Determination (R2)
r2ANN2 = r2_score(y_test2, y_predANN2)

# Calcular Mean Absolute Error (MAE)
maeANN2 = mean_absolute_error(y_test2, y_predANN2)

# Imprimir los resultados
print("Coefficient of Determination (R2):", r2ANN2)
print("Mean Absolute Error (MAE):", maeANN2)

25/25 [==============================] - 0s 3ms/step
Coefficient of Determination (R2): -2.532922164660907


NameError: name 'maeANN2' is not defined

Sigue siendo bastante malo el modelo.

## 3. Random Forest (6k)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Crear y entrenar el modelo RandomForestRegressor
random_forest_reg = RandomForestRegressor(n_estimators=500, random_state=42)
random_forest_reg.fit(X_train_scale, y_train)

# Hacer predicciones con el modelo entrenado
y_pred_rf = random_forest_reg.predict(X_train_scale)

# Calcular Coefficient of Determination (R2)
r2_rf_reg = r2_score(y_test, y_pred_rf_reg)

# Calcular Mean Absolute Error (MAE)
mae_rf_reg = mean_absolute_error(y_test, y_pred_rf_reg)

# Calcular Root Mean Squared Error (RMSE)
rmse_rf_reg = np.sqrt(mean_squared_error(y_test, y_pred_rf_reg))

# Calcular Relative Absolute Error (RAE)
rae_rf_reg = np.mean(np.abs(y_pred_rf_reg - y_test)) / np.mean(np.abs(y_test - np.mean(y_test)))

# Calcular Root Relative Squared Error (RRSE)
rrse_rf_reg = np.sqrt(np.sum(np.square(y_pred_rf_reg - y_test))) / np.sqrt(np.sum(np.square(y_test - np.mean(y_test))))

# Imprimir los resultados
print("Coefficient of Determination (R2):", r2_rf_reg)
print("Mean Absolute Error (MAE):", mae_rf_reg)
print("Relative Absolute Error (RAE):", rae_rf_reg)
print("Root Relative Squared Error (RRSE):", rrse_rf_reg)
print("Root Mean Squared Error (RMSE):", rmse_rf_reg)



Accuracy: 0.823583180987203
Recall: 0.04081632653061224
Precision: 0.6153846153846154
F1 Score: 0.07655502392344496
Confusion Matrix:
 [[893   5]
 [188   8]]


# 4. Comparamos los resultados de los modelos

In [14]:
# Crear un diccionario con los datos
data = {
    'Modelo': ['SVM', 'ANN', 'RF'],
    'Accuracy': [accuracySVM, accuracyANN, accuracyRF],
    'Precision': [precisionSVM, precisionANN, precisionRF],
    'Recall': [recallSVM, recallANN, recallRF],
    'F1 Score': [f1SVM, f1ANN, f1RF]
}

# Crear un DataFrame con los datos
comparacion = pd.DataFrame(data)

# Mostrar la tabla
print(comparacion)

  Modelo  Accuracy  Precision    Recall  F1 Score
0    SVM  0.823583   0.588235  0.051020  0.093897
1    ANN  0.813528   0.409091  0.091837  0.150000
2     RF  0.823583   0.615385  0.040816  0.076555


- Precisión (Accuracy):
    - La precisión es la proporción de predicciones correctas sobre el total de predicciones.
    - Una precisión del 1.0 indica que todas las predicciones son correctas, mientras que una precisión del 0.0 indica que ninguna predicción es correcta.
    - Es una métrica general del rendimiento del modelo, pero puede ser engañosa si hay un desequilibrio en las clases objetivo.

- Recall (Exhaustividad):
    - La exhaustividad es la proporción de positivos reales que se identificaron correctamente.
    - Una exhaustividad del 1.0 indica que todas las instancias positivas se han identificado correctamente, mientras que una exhaustividad del 0.0 indica que ninguna instancia positiva se ha identificado correctamente.
    - Es útil cuando la identificación de instancias positivas es crítica y no se pueden permitir falsos negativos.

- Precisión (Precision):
    - La precisión es la proporción de instancias positivas predichas que fueron correctamente identificadas.
    - Una precisión del 1.0 indica que todas las instancias predichas como positivas son verdaderas positivas, mientras que una precisión del 0.0 indica que ninguna instancia predicha como positiva es realmente positiva.
    - Es útil cuando es importante evitar falsos positivos.

- F1 Score:
    - El puntaje F1 es la media armónica de precisión y exhaustividad.
    - Proporciona un equilibrio entre precisión y exhaustividad, lo que lo hace útil cuando se desea tener un buen rendimiento en ambas métricas.
    - Un puntaje F1 del 1.0 indica un equilibrio perfecto entre precisión y exhaustividad.

- Matriz de Confusión:
    - La matriz de confusión es una tabla que describe la calidad de las predicciones del modelo.
    - Proporciona una descripción detallada de los resultados de clasificación, mostrando el número de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos.
    - Es útil para identificar dónde el modelo está cometiendo errores y para evaluar el desempeño en cada clase por separado.